# Setup

In [2]:
# Import packages we'll need later on in this tutorial
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import BW25 packages. You'll notice the packages are imported individually, unlike a one-and-done import with BW2.
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

In [3]:
bd.projects

Brightway2 projects manager with 2 objects:
	autumn-school
	default
Use `projects.report()` to get a report on all projects.

In [4]:
# You need to set a project. Give it a name!
name = "autumn-school"
bd.projects.set_current(name)

In [5]:
# If this is your first time using this BW25 project, this should be an empty dictionary! 
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

In [6]:
# If this is the first time you've set up the project, you need to install the biosphere flows.
# You do not need to run this subsequent times.

bi.bw2setup() ## THIS YIELDS AN ERROR AT THE MOMENT

Biosphere database already present!!! No setup is needed


In [7]:
# Import the path where your EI database is stored. 
# Note that the EI database must be unzipped and the path should end at the datasets folder.
ei_path = r"C:\Users\pierr\cutoff39\datasets"

# You will also need to give your database a name. This name will appear when you call bd.databases.
# Here, I am using EI v3.9.1 cutoff.
ei_name = "ecoinvent-391-cutoff"

# When we execute this cell, we will check if it's already been imported, and if not (else) we import it.

if ei_name in bd.databases:
    print("Database has already been imported.")
else:
# Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()

Database has already been imported.


In [7]:
# Import the path where your EI database is stored. 
# Note that the EI database must be unzipped and the path should end at the datasets folder.
ei_path = r"C:\Users\pierr\cutoff39\datasets"

# You will also need to give your database a name. This name will appear when you call bd.databases.
# Here, I am using EI v3.9.1 cutoff.
ei_name = "ecoinvent-391-cutoff"

# When we execute this cell, we will check if it's already been imported, and if not (else) we import it.

if ei_name in bd.databases:
    print("Database has already been imported.")
else:
# Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()

Database has already been imported.


In [8]:
ei=bd.Database("ecoinvent-391-cutoff")

In [31]:
biodb=bd.Database('biosphere3')

In [32]:
print(biodb)

Brightway2 SQLiteBackend: biosphere3


In [9]:
print([act for act in ei if "steel production, electric, low-alloyed" == act['name'] and "Europe without Switzerland and Austria" == act['location']])

['steel production, electric, low-alloyed' (kilogram, Europe without Switzerland and Austria, None)]


In [10]:
activity=[act for act in ei if "transport, freight, lorry 16-32 metric ton, EURO6" == act['name'] and "RER" == act['location']][0]
activity.as_dict()['reference product']

'transport, freight, lorry 16-32 metric ton, EURO6'

# Importer from Pierre's routine

In [23]:
ei.new_activity

Signature: ei.new_activity(code, **kwargs)
Docstring: <no docstring>
Source:   
    def new_activity(self, code, **kwargs):
        return self.new_node(code, **kwargs)
File:      c:\users\pierr\anaconda3\envs\bw\lib\site-packages\bw2data\backends\base.py
Type:      method

In [89]:
"""
Waste must be used in exchanges using -1 factor
"""

def process_creation(code,name,unit,location,Dexchanges):
    print('->process_creation()')
    #The following for-loop is only to guarantee that we can create a process
    for activity in [act for act in ei if name == act['name'] and location == act['location']]:
        activity.delete()
    
    activity = ei.new_activity(**{
        'activity': name,
        'activity type': 'process',
        'code': code,
        'location': location,
        'name':name,
        'production amount': 1.0,
        'unit': unit,
        'reference product': name,
    })
    activity.save()
    print('   len(Dexchanges[input name]) : '+str(len(Dexchanges['input name'])))
    print('   len(Dexchanges[input location]) : '+str(len(Dexchanges['input location'])))
    print('   len(Dexchanges[amount]) : '+str(len(Dexchanges['amount'])))
    print('   len(Dexchanges[unit]) : '+str(len(Dexchanges['unit'])))
    print('   len(Dexchanges[type]) : '+str(len(Dexchanges['type'])))
    print('   len(Dexchanges[uncertainty]) : '+str(len(Dexchanges['uncertainty'])))
    for i in range (0,len(Dexchanges['input name'])):
        act_input=[act for act in ei if Dexchanges['input name'][i] == act['name'] and Dexchanges['input location'][i] == act['location'] and Dexchanges['unit'][i] in act['unit']]
        #print(act_input)
        if act_input==[]:
            print('ERROR : input for exchange not found : '+str(Dexchanges['input name'][i])+"  "+str(Dexchanges['input location'][i]))
        act_exchange=activity.new_exchange(input=act_input[0],amount=Dexchanges['amount'][i],unit=Dexchanges['unit'][i],type=Dexchanges['type'][i])
        uncertainty=Dexchanges['uncertainty'][i]
        for key in uncertainty.keys():
            act_exchange[key]=Dexchanges['uncertainty'][i][key]
        act_exchange.save()
        activity.save()
    return activity


In [48]:
LHV_diesel = 42.8 #low heat value of diesel in MJ/kg
p_diesel = 0.84 #density of diesel in kg/L

"""
--------------RCC, crushed, operation------------

Process for recycling reinforced concrete.
Following the work of Marjan et al 2020

Description : 
Adapted from the Swiss process.
Errors corrected: there was both a call for an infrastructure flow (gravel/sand quarry infrstructure) and some intermediate flows also concerning infrastructure.
Concerned flows are : building hall, conveyor belt, industrial machine, elementary flows concerning transformation and occupation, recultivation
All these flows were added to the "gravel/sand quarry construction" process by multiplying them by 4.75E-11 (the value of the infrastructure flow.

Furthermore, there is also a process callaed gravel and sans quarry operation, but for the product "gravel round" that is not a construction material (no crushers in the process).
I have remodeled the process by calling one infrastructure flow and one operation flow.
Then I have remodeled "Quarry construction" and "quarry operation processes".

"""

Line = 3 #number of lines of the quarry process
ratioA1 = 0.377 #proportion of A1 in the total production of quarry process
ratioA2 = 0.187 #proportion of A2 in the total production of quarry process
ratioA3 = 0.454 #proportion of A3 in the total production of quarry process
AnnualProd = 20000 #☻kg/year annual production
NbWeek = 3 #week/year number of weeks with mobile crusher
R = 1.35 
TimeRatio = NbWeek/54 #time of crushing in the total working annual time
IR = TimeRatio /AnnualProd #ratio of infrastructure for 1kg of crushed material

D_exchanges_RCC={
    "input name": ['market for diesel, burned in building machine',
                   'market for electricity, medium voltage',
                   'market for gravel/sand quarry infrastructure',
                   'market for heat, central or small-scale, other than natural gas',
                   'market for lubricating oil',
                   'market for steel, low-alloyed, hot rolled',
                   'market for synthetic rubber',
                   'market for tap water'
                   ]
    ,"input location" : ['GLO',
                         'FR',
                         'GLO',
                         'Europe without Switzerland',
                         'RER',
                         'GLO',
                         'GLO',
                         'Europe without Switzerland'
                         
                         ]
    ,"amount" : [0.0143/Line ,
                 0.00906/Line ,
                 4.75E-11/Line *IR,
                 0.00491/Line ,
                 2.5E-6/Line ,
                 5.1E-5/Line ,
                 4.0E-6/Line ,
                 1.0/Line ,
                 
                 ]
    ,"unit" :["megajoule",
              "kilowatt hour",
              "unit",
              "megajoule",
              "kilogram",
              "kilogram",
              "kilogram",
              "kilogram",
              ]
    ,"type":["technosphere",
             "technosphere",
             "technosphere",
             "technosphere",
             "technosphere",
             'technosphere',
             'technosphere',
             'technosphere']
    }



D_exchanges_RCC['uncertainty']=[]
for amount in D_exchanges_RCC['amount'] :
    D_exchanges_RCC['uncertainty'].append({'uncertainty type' : 1, 'loc' : amount})
    
RCC_crushed=process_creation('RCC, crushed, operation','RCC, crushed, operation','kilogram','FR',D_exchanges_RCC)
print(D_exchanges_RCC)

activity=RCC_crushed

act_input=[act for act in biodb if 'Water, unspecified natural origin' in act['name'] and 'in water' in act['categories']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=0.001111945/Line ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=0.001111945
act_exchange['scale']=1.1051709180756477
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'market for municipal solid waste' in act['name'] and 'FR' in act['location']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=-2.12E-6/Line  ,type='biosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()
"""
act_input=[act for act in biodb if 'Particulates, < 2.5 um' in act['name'] and 'low population density, long-term' in act['categories']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=4.0E-10/Line   ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=4.0E-10
act_exchange['scale']=1.7915751222543432
act_exchange.save()
activity.save()

act_input=[act for act in biodb if 'Particulates, > 10 um' in act['name'] and 'low population density, long-term' in act['categories']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=5.6E-9/Line    ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=5.6E-9
act_exchange['scale']=1.3268964411453439
act_exchange.save()
activity.save()

act_input=[act for act in biodb if 'Particulates, > 2.5 um, and < 10um' in act['name'] and 'low population density, long-term' in act['categories']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=2.0E-9/Line   ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=2.0E-9
act_exchange['scale']=1.4918246976412703
act_exchange.save()
activity.save()
"""
act_input=[act for act in ei if 'market for waste mineral oil' in act['name'] and 'Europe without Switzerland' in act['location']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=-2.5E-6/Line   ,type='biosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in biodb if 'Water' in act['name'] and 'air' in act['categories'] and len(act['categories'])==1]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=0.306585/1000/Line    ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=3.06585E-4
act_exchange['scale']=1.2214027581601699
act_exchange.save()
activity.save()

act_input=[act for act in biodb if 'Water' in act['name'] and 'water' in act['categories'] and len(act['categories'])==1]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=8.1756E-4/Line ,type='biosphere')
act_exchange['uncertainty type']=2
act_exchange['loc']=8.1756E-4
act_exchange['scale']=1.2214027581601699
act_exchange.save()
activity.save()

->process_creation()
   len(Dexchanges[input name]) : 8
   len(Dexchanges[input location]) : 8
   len(Dexchanges[amount]) : 8
   len(Dexchanges[unit]) : 8
   len(Dexchanges[type]) : 8
   len(Dexchanges[uncertainty]) : 8
{'input name': ['market for diesel, burned in building machine', 'market for electricity, medium voltage', 'market for gravel/sand quarry infrastructure', 'market for heat, central or small-scale, other than natural gas', 'market for lubricating oil', 'market for steel, low-alloyed, hot rolled', 'market for synthetic rubber', 'market for tap water'], 'input location': ['GLO', 'FR', 'GLO', 'Europe without Switzerland', 'RER', 'GLO', 'GLO', 'Europe without Switzerland'], 'amount': [0.0047666666666666664, 0.00302, 4.3981481481481477e-17, 0.0016366666666666667, 8.333333333333334e-07, 1.7e-05, 1.3333333333333332e-06, 0.3333333333333333], 'unit': ['megajoule', 'kilowatt hour', 'unit', 'megajoule', 'kilogram', 'kilogram', 'kilogram', 'kilogram'], 'type': ['technosphere', 'tech

In [78]:
act=[act for act in ei if 'concrete production' in act['name'] and 'CH'in act['location']]
for a in act:
    print(a)

'unreinforced concrete production, 15MPa, with cement, CEM II/A' (cubic meter, CH, None)
'concrete production, 25MPa, for building construction, for interior use, with cement ZN/D' (cubic meter, CH, None)
'concrete production, 30MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates' (kilogram, CH, None)
'concrete production, 25MPa, for building construction, with cement, CEM II/B' (cubic meter, CH, None)
'concrete production, 30MPa, for drilled piles, with cement, CEM II/B' (cubic meter, CH, None)
'concrete production, 30MPa, for building construction, for interior use, with cement ZN/D, with 30% RC-M aggregates' (kilogram, CH, None)
'unreinforced concrete production, 15MPa, with cement, CEM II/B' (cubic meter, CH, None)
'treatment of sewage sludge, 97% water, WWT, WW from concrete production, landfarming' (kilogram, CH, None)
'lean concrete production, with cement CEM II/A' (cubic meter, CH, None)
'concrete production, 30MPa, for drilled piles, w

In [81]:
act=[act for act in ei if 'cement production, CEM II/B' in act['name'] and 'CH' in act['location']][0]
for exch in act.exchanges():
    print(exch)
    print(exch['name'])
    print(exch['amount'])

Exchange: 1.0 kilogram 'cement production, CEM II/B' (kilogram, CH, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
cement, CEM II/B
1.0
Exchange: 0.136 kilogram 'burnt shale production' (kilogram, DE, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
burnt shale
0.136
Exchange: 2.73e-11 unit 'cement factory construction' (unit, CH, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
cement factory
2.73e-11
Exchange: 0.688 kilogram 'clinker production' (kilogram, CH, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
clinker
0.688
Exchange: 0.0342 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, CH, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
electricity, medium voltage
0.0342
Exchange: 0.00021 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/B' (kilogram, CH, None)>
ethylene glycol
0.00021
Exchange: 0.0211 kilogram 'market for gypsum, mineral' (kilogram, RER, None

In [87]:
"""SHEET STEEL"""
"""
Hypothesis : INIES : https://www.base-inies.fr/iniesV4/dist/consultation.html?id=32988

A1-A3
Primary steel : 80%
Secondary steel : 20%
Chutes : 0,0887 kg/kg

A4-A5 
Distance chantier route : 593 km
Distance chantier barge : 13.5 km
Intrant acier supplémentaires : 0.92%

Consommation d’énergie pour les engins de levage :
- Electricité : 2,89E-04 kWh/kg de produit
- Gazole non routier : 1,12E-01 MJ/kg de produit
- Propane ou GPL : 1,53E-03 MJ/kg de produit
Consommation d’énergie pour le boulonnage :
- Electricité : 1,16E-04 kWh/kg de produit

C1-C4
88% recyclé -> 375 km
11% réutilisation -> 375 km
1% landfill -> 150 km

"""

chutes=0.0887
acier_supp=0.0092
tot_acier=1+0.0843+0.0206

#--------------Activity Creation

for activity in [act for act in ei if 'sheet steel' == act['name'] and 'RER' == act['location']]:
    activity.delete()
activity =ei.new_activity(**{
        'activity': 'sheet steel',
        'activity type': 'process',
        'code': 'sheet_steel',
        'location': 'RER',
        'name':'sheet steel',
        'production amount': 1.0,
        'unit': 'kg',
        'reference product': 'sheet_steel',
    }) 
activity.save()

#-------------Exchanges

#A1-A3
act_input=[act for act in ei if 'steel production, electric, low-alloyed' in act['name'] and 'Europe without Switzerland and Austria' in act['location']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=0.20*tot_acier  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'steel production, converter, low-alloyed' in act['name'] and 'RER' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=0.80*tot_acier  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'hot rolling, steel' in act['name'] and 'Europe without Austria' in act['location']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=1  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

#A4-A5
act_input=[act for act in ei if 'transport, freight, lorry 16-32 metric ton, EURO6' in act['name'] and 'RER' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=593/1000  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'market for diesel, burned in building machine' in act['name'] and 'GLO' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=1.12E-01  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'market for electricity, medium voltage' in act['name'] and 'FR' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=2.89E-04  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'market for electricity, medium voltage' in act['name'] and 'FR' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=1.16E-04  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()


#C1-C4
act_input=[act for act in ei if 'transport, freight, lorry 16-32 metric ton, EURO6' in act['name'] and 'RER' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=0.88*375/1000  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'transport, freight, lorry 16-32 metric ton, EURO6' in act['name'] and 'RER' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=0.11*375/1000  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'transport, freight, lorry 16-32 metric ton, EURO6' in act['name'] and 'RER' in act['location']]
act_exchange=activity.new_exchange(input=act_input[0],amount=0.01*150/1000  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()

act_input=[act for act in ei if 'treatment of scrap steel, inert material landfill' in act['name'] and 'CH' in act['location']]
for act in act_input:
    print(act)
print(act_input[0])
act_exchange=activity.new_exchange(input=act_input[0],amount=-0.88  ,type='technosphere')
act_exchange['uncertainty type']=1
act_exchange['loc']=act_exchange['amount']
act_exchange.save()
activity.save()
print('done')

'steel production, electric, low-alloyed' (kilogram, Europe without Switzerland and Austria, None)
'steel production, electric, low-alloyed' (kilogram, Europe without Switzerland and Austria, None)
'hot rolling, steel' (kilogram, Europe without Austria, None)
'hot rolling, steel' (kilogram, Europe without Austria, None)
'treatment of scrap steel, inert material landfill' (kilogram, CH, None)
'treatment of scrap steel, inert material landfill' (kilogram, CH, None)


NameError: name 'done' is not defined

In [90]:
"""
--------------DC-concrete------------

Concrete CEMII based on INIES 11081 
https://www.base-inies.fr/iniesV4/dist/consultation.html?id=11081


"""
d_A4 = 18.5 #distance transport au site
d_C2_land = 30 #Distance pour élimination déchets
d_C2_recycle = 30 #Distance pour recyclage
p_concrete = 2400 #masse volumique béton
per_landfill = 0.25 #Pourcentage enfouissement
per_recycled = 0.75 #Pourcentage recyclé

D_exchanges_DC_concrete={
    "input name": ['concrete production, 25MPa, for building construction, with cement, CEM II/B',
                   'electricity, high voltage, production mix',
                   'transport, freight, lorry 16-32 metric ton, EURO6',
                   'transport, freight, lorry 16-32 metric ton, EURO6',
                   'transport, freight, lorry 16-32 metric ton, EURO6',
                   'treatment of waste concrete, inert material landfill',
                   'RCC, crushed, operation']
    ,"input location" : ['CH',
                         'FR',
                         'RER',
                         'RER',
                         'RER',
                         'Europe without Switzerland',
                         'FR']
    ,"amount" : [1.03, #03 pour 3% de perte
                 385.2/3.6, #A5 installation
                 p_concrete*d_A4/1000, #A4
                 p_concrete*d_C2_land *per_landfill/1000, #C2
                 p_concrete*d_C2_recycle *per_recycled/1000, #C2
                 -1*per_landfill *p_concrete,
                 per_recycled*p_concrete]
    ,"unit" :["cubic meter",
              "kilowatt hour",
              "ton kilometer",
              "ton kilometer",
              'ton kilometer',
              'kilogram',
              'kilogram']
    ,"type":["technosphere",
             "technosphere",
             "technosphere",
             "technosphere",
             "technosphere",
             'technosphere',
             'technosphere']
    }

D_exchanges_DC_concrete['uncertainty']=[]
for amount in D_exchanges_DC_concrete['amount'] :
    D_exchanges_DC_concrete['uncertainty'].append({'uncertainty type' : 1, 'loc' : amount})
    
DC_concrete=process_creation('concrete_production_cemII','concrete production CEMII','cubic meter','RER',D_exchanges_DC_concrete)
print(DC_concrete)


->process_creation()
   len(Dexchanges[input name]) : 7
   len(Dexchanges[input location]) : 7
   len(Dexchanges[amount]) : 7
   len(Dexchanges[unit]) : 7
   len(Dexchanges[type]) : 7
   len(Dexchanges[uncertainty]) : 7
'concrete production CEMII' (cubic meter, RER, None)


In [91]:
#double check
act=[act for act in ei if 'RCC, crushed, operation' in act['name']]
for a in act:
    print(a)
act=[act for act in ei if 'sheet steel' in act['name']]
for a in act:
    print(a)
act=[act for act in ei if 'concrete production CEMII' in act['name']]
for a in act:
    print(a)

'RCC, crushed, operation' (kilogram, FR, None)
'sheet steel' (kg, RER, None)
'concrete production CEMII' (cubic meter, RER, None)


# Create cement processes

1. cement production, CEM II/A-S

In [191]:
act=[act for act in ei if 'cement production, CEM II/A' in act['name'] and 'Europe without Switzerland'in act['location']][0]
print(act)

'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)
0
Exchange: 1.0 kilogram 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.002 kilogram 'market for burnt shale' (kilogram, GLO, None) to 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)>
2
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.8265 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.0374 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00021 kilogram 'market for ethylene glycol' (k

In [167]:
new_act=act.copy(name='cement production, CEM II/A-S',code='cement production, CEM II/A-S')
new_act.save()

In [181]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.8265 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0374 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.00021 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.003136 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00011 kilogram 'market for steel, low-alloyed' (kilogram, GLO,

In [169]:
exs[8].delete()
exs[6].delete()
exs[5].delete()
exs[0].delete()

In [170]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.8265 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0374 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.00021 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.003136 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00011 kilogram 'market for steel, low-alloyed' (kilogram, GLO,

In [179]:
search=[sear for sear in ei if 'market for ground granulated blast furnace slag' in sear['name'] and 'RoW' in sear['location']][0]
print(search)

'market for ground granulated blast furnace slag' (kilogram, RoW, None)


In [184]:
new_act.new_edge(input=search,amount=0.135,type='technosphere').save()

2. cement production, CEM II/A-V

In [200]:
act=[act for act in ei if 'cement production, CEM II/A' ==act['name'] and 'Europe without Switzerland'in act['location']][0]
print(act)

'cement production, CEM II/A' (kilogram, Europe without Switzerland, None)


In [206]:
new_act=act.copy(name='cement production, CEM II/A-V',code='cement production, CEM II/A-V')
new_act.save()

In [207]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.002 kilogram 'market for burnt shale' (kilogram, GLO, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
1
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.8265 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.0374 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.00021 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.0098 kilogram 'market for ground granulated blast furnace slag' (kilogram, RoW, None) to 'cement production, CEM II/A-V' (kilogram,

In [208]:
exs[8].delete()
exs[6].delete()
exs[5].delete()
exs[0].delete()

In [209]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.8265 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0374 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.00021 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.003136 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00011 kilogram 'market for steel, low-alloyed' (kilogram, GLO,

In [210]:
search=[sear for sear in ei if 'market for fly ash and scrubber sludge' in sear['name'] and 'Europe without Switzerland' in sear['location']][0]
print(search)

'market for fly ash and scrubber sludge' (kilogram, Europe without Switzerland, None)


In [211]:
new_act.new_edge(input=search,amount=0.16875,type='technosphere').save() #0.135*1.25


3. cement production, CEM II/B-S


In [212]:
act=[act for act in ei if 'cement production, CEM II/B' in act['name'] and 'Europe without Switzerland'in act['location']][0]
print(act)

'cement production, CEM II/B' (kilogram, Europe without Switzerland, None)


In [213]:
new_act=act.copy(name='cement production, CEM II/B-S',code='cement production, CEM II/B-S')
new_act.save()

In [214]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.0042 kilogram 'market for burnt shale' (kilogram, GLO, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
1
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.684 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.0365 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.00031 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.021 kilogram 'market for ground granulated blast furnace slag' (kilogram, RoW, None) to 'cement production, CEM II/B-S' (kilogram, 

In [215]:
exs[8].delete()
exs[6].delete()
exs[5].delete()
exs[0].delete()

In [216]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.684 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0365 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.00031 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.00672 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00011 kilogram 'market for steel, low-alloyed' (kilogram, GLO, N

In [217]:
search=[sear for sear in ei if 'market for ground granulated blast furnace slag' in sear['name'] and 'RoW' in sear['location']][0]
print(search)

'market for ground granulated blast furnace slag' (kilogram, RoW, None)


In [218]:
new_act.new_edge(input=search,amount=0.316,type='technosphere').save()

4. cement production, CEM II/B-V

In [219]:
act=[act for act in ei if 'cement production, CEM II/B' ==act['name'] and 'Europe without Switzerland'in act['location']][0]
print(act)

'cement production, CEM II/B' (kilogram, Europe without Switzerland, None)


In [220]:
new_act=act.copy(name='cement production, CEM II/B-V',code='cement production, CEM II/B-V')
new_act.save()

In [221]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.0042 kilogram 'market for burnt shale' (kilogram, GLO, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
1
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.684 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.0365 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.00031 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.021 kilogram 'market for ground granulated blast furnace slag' (kilogram, RoW, None) to 'cement production, CEM II/B-V' (kilogram, 

In [222]:
exs[8].delete()
exs[6].delete()
exs[5].delete()
exs[0].delete()

In [223]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 5.36e-11 unit 'market for cement factory' (unit, GLO, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.684 kilogram 'clinker production' (kilogram, Europe without Switzerland, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0365 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.00031 kilogram 'market for ethylene glycol' (kilogram, GLO, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.00672 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.00011 kilogram 'market for steel, low-alloyed' (kilogram, GLO, N

In [224]:
search=[sear for sear in ei if 'market for fly ash and scrubber sludge' in sear['name'] and 'Europe without Switzerland' in sear['location']][0]
print(search)

'market for fly ash and scrubber sludge' (kilogram, Europe without Switzerland, None)


In [225]:
new_act.new_edge(input=search,amount=0.395,type='technosphere').save() #0.316*1.25

5. market for cement, adjusted

In [226]:
act=[act for act in ei if 'cement, all types to generic market for cement, unspecified' ==act['name'] and 'Europe without Switzerland'in act['location']][0]
print(act)

'cement, all types to generic market for cement, unspecified' (kilogram, Europe without Switzerland, None)


In [227]:
new_act=act.copy(name='market for cement, all types, adjusted',code='market for cement, all types, adjusted')
new_act.save()

In [228]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.4337506 kilogram 'market for cement, CEM II/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.0459449 kilogram 'market for cement, CEM II/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.1370168 kilogram 'market for cement, CEM III/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.0141742 kilogram 'market for cement, CEM III/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.0023624 kilogram 'market for cement, CEM III/C' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.017

In [234]:
exs[1].delete()
exs[0].delete()

In [235]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.1370168 kilogram 'market for cement, CEM III/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.0141742 kilogram 'market for cement, CEM III/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0023624 kilogram 'market for cement, CEM III/C' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.017132 kilogram 'market for cement, CEM IV/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.0019036 kilogram 'market for cement, CEM IV/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.0277

In [236]:
search=[sear for sear in ei if 'cement production, CEM II/A-S' in sear['name'] and 'Europe without Switzerland'in sear['location']][0]
print(search)
new_act.new_edge(input=search,amount=0.43375/2,type='technosphere').save()

search=[sear for sear in ei if 'cement production, CEM II/A-V' in sear['name'] and 'Europe without Switzerland'in sear['location']][0]
print(search)
new_act.new_edge(input=search,amount=0.43375/2,type='technosphere').save()

search=[sear for sear in ei if 'cement production, CEM II/B-S' in sear['name'] and 'Europe without Switzerland'in sear['location']][0]
print(search)
new_act.new_edge(input=search,amount=0.0459449/2,type='technosphere').save()

search=[sear for sear in ei if 'cement production, CEM II/B-V' in sear['name'] and 'Europe without Switzerland'in sear['location']][0]
print(search)
new_act.new_edge(input=search,amount=0.0459449/2,type='technosphere').save()

'cement production, CEM II/A-S' (kilogram, Europe without Switzerland, None)
'cement production, CEM II/A-V' (kilogram, Europe without Switzerland, None)
'cement production, CEM II/B-S' (kilogram, Europe without Switzerland, None)
'cement production, CEM II/B-V' (kilogram, Europe without Switzerland, None)


In [237]:
exs = [ex for ex in new_act.technosphere()]
for i,ex in enumerate(exs):
    print(i)
    print(ex)

0
Exchange: 0.1370168 kilogram 'market for cement, CEM III/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
1
Exchange: 0.0141742 kilogram 'market for cement, CEM III/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
2
Exchange: 0.0023624 kilogram 'market for cement, CEM III/C' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
3
Exchange: 0.017132 kilogram 'market for cement, CEM IV/A' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
4
Exchange: 0.0019036 kilogram 'market for cement, CEM IV/B' (kilogram, Europe without Switzerland, None) to 'market for cement, all types, adjusted' (kilogram, Europe without Switzerland, None)>
5
Exchange: 0.0277

# Importer from CSV DOESN T WORK

In [20]:
activity.as_dict()['reference product']

'transport, freight, lorry 16-32 metric ton, EURO6'

In [12]:
bi.CSVImporter?

Init signature: bi.CSVImporter(filepath)
Docstring:      Generic CSV importer
Init docstring:
Initialize the ImportBase object.

Parameters
----------
*args :
    Variable length argument list.
**kwargs :
    Arbitrary keyword arguments.

Raises
------
NotImplemented :
    This class should be subclassed.
File:           c:\users\pierr\anaconda3\envs\bw\lib\site-packages\bw2io\importers\excel.py
Type:           type
Subclasses:     

In [21]:
i=bi.CSVImporter(r"C:/Users/pierr/autumn-school-bw2/lci-reinforcing-steel.csv")
i.apply_strategies()
i.match_database(fields=["name", "location", "unit"])
i.match_database("ecoinvent-391-cutoff", fields=["name", "reference product", "unit", "location"])
i.match_database("biosphere3", fields=["name", "categories"])
i.statistics()
#i.drop_unlinked(i_am_reckless=True)
i.write_database()

Extracted 1 worksheets in 0.00 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 35.73 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [14]:
activity=[act for act in ei if "reinforcing steel production" == act['name'] and "RER" == act['location']][0]

IndexError: list index out of range